In [8]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Ruta al archivo Parquet
ruta_archivo_parquet = r'C:\Users\guard\OneDrive\Desktop\Henry Data Science\Proyecto-FInal\Datos\gm_rev_NJNY_202502182030.parquet'

# Leer el archivo Parquet en un DataFrame
df = pd.read_parquet(ruta_archivo_parquet)

# Asegurarnos de que no hay valores nulos en la columna 'text'
df = df.dropna(subset=['text'])

# Inicializar el analizador de sentimientos VADER
analyzer = SentimentIntensityAnalyzer()

# Definir una función para calcular la puntuación de sentimientos
def analizar_sentimiento(texto):
    if texto is None:
        texto = ""  # Reemplazar valores nulos con cadena vacía
    sentimientos = analyzer.polarity_scores(texto)
    return sentimientos['compound']

# Aplicar la función a la columna 'text' para obtener la puntuación de sentimientos
df['sentimiento_puntuacion'] = df['text'].apply(analizar_sentimiento)


In [3]:
# Crear una columna 'sentimiento' con valores categóricos basado en la puntuación de sentimientos
df['sentimiento'] = df['sentimiento_puntuacion'].apply(lambda x: 'positivo' if x > 0 else 'negativo' if x < 0 else 'neutro')

In [ ]:
# Inicializar el vectorizador TF-IDF
vectorizer = TfidfVectorizer()

# Definir las características y etiquetas para el entrenamiento
X_train_tfidf = vectorizer.fit_transform(df['text'])
y_train = df['sentimiento']

# Mostrar la distribución inicial de clases
print("Distribución inicial de clases:", Counter(y_train))

# Aplicar SMOTE para realizar el sobremuestreo
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_tfidf, y_train)

# Mostrar la nueva distribución de clases
print("Distribución después de SMOTE:", Counter(y_train_res))


Distribución inicial de clases: Counter({'positivo': 76489, 'negativo': 8269, 'neutro': 5992})
Distribución después de SMOTE: Counter({'positivo': 76489, 'negativo': 76489, 'neutro': 76489})


In [13]:
from sklearn.linear_model import LogisticRegression

# Definir los parámetros de la cuadrícula
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],  # Normas de penalización L1 y L2
    'solver': ['liblinear']  # Solvers compatibles con L1 y L2
}

# Inicializar el modelo
modelo = LogisticRegression()

# Configurar la búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=5, scoring='f1_weighted', n_jobs=-1)

# Entrenar el modelo con la búsqueda en cuadrícula
grid_search.fit(X_train_res, y_train_res)

# Obtener los mejores parámetros
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)


Mejores parámetros: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}


In [16]:
from sklearn.metrics import classification_report

# Entrenar el modelo con los mejores parámetros
modelo_final = LogisticRegression(**best_params)
modelo_final.fit(X_train_res, y_train_res)

# Dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentimiento'], test_size=0.2, random_state=42)

# Transformar los datos de prueba usando el vectorizador TF-IDF
X_test_tfidf = vectorizer.transform(X_test)

# Entrenar el modelo con los mejores parámetros
modelo_final = LogisticRegression(**best_params)
modelo_final.fit(X_train_res, y_train_res)

# Evaluar el modelo final
y_pred = modelo_final.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

    negativo       0.83      0.96      0.89      1633
      neutro       0.89      0.98      0.93      1130
    positivo       1.00      0.97      0.98     15387

    accuracy                           0.97     18150
   macro avg       0.90      0.97      0.93     18150
weighted avg       0.97      0.97      0.97     18150



In [19]:
import joblib

# Guardar el modelo final y el vectorizador
joblib.dump(modelo_final, 'modelo_sentimientos_final.pkl')
joblib.dump(vectorizer, 'vectorizador_tfidf.pkl')


['vectorizador_tfidf.pkl']